In [ ]:
# #Using yfinance module to get the data from yahoo finance
# from datetime import datetime
# from datetime import timedelta
# from pandas_datareader import data as pdr
# import pandas as pd
# import matplotlib.pyplot as plt

import yfinance as yf
import pandas as pd
from datetime import date
pd.set_option("display.max_rows", None, "display.max_columns", None)
import pandas_ta as ta



In [ ]:
#import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
print(msft.info)

# get historical market data
hist = msft.history(period="5d")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# get option chain for specific expiration
#opt = msft.option_chain('YYYY-MM-DD')
# data available via: opt.calls, opt.puts

In [ ]:
#info in the table
ticker=yf.Ticker('AAPL')
df=ticker.history(period='52wk',back_adjust=True)
print(f"Length of data-frame {len(df.index)}")

In [ ]:
print(f"max from df {df['High'].max().round(2)}")
print(f"min from df {df['Low'].min().round(2)}")
print(f"From yfinance 52 week high: {ticker.info['fiftyTwoWeekHigh']}")
print(f"From yfinance 52 week low:  {ticker.info['fiftyTwoWeekLow']}")

In [ ]:
info_df=pd.DataFrame.from_dict(msft.info,orient="index",columns=['INFO'])
#info_df=info_df.sort_values(by=['index'])
info_df
#info_df.reset_index()


In [ ]:
f=['META']
for ticker in f:
    tick = yf.Ticker(ticker)
    
    print(f"Volume of {ticker}: {tick.info['floatShares']/10**6}")
    

In [ ]:
root="/home/thakur/stock_information/"
save_dir="/home/thakur/test/combined/"
source_dir="/home/thakur/test/"



df_rs=pd.read_csv('https://raw.githubusercontent.com/skyte/rs-log/main/output/rs_stocks.csv')   
def get_rs_value(ticker):
    if ticker in list(df_rs['Ticker']):
        return (df_rs.loc[df_rs['Ticker']==ticker]['Percentile'].values[0])
    else: 
        return 0
    
# def get_sector(ticker):
#     if ticker in list(df_rs['Ticker']):
#         return (df_rs.loc[df_rs['Ticker']==ticker]['Percentile'].values[0])
#     else: 
#         return 0
    
# def get_industry(ticker):
#     if ticker in list(df_rs['Ticker']):
#         return (df_rs.loc[df_rs['Ticker']==ticker]['Percentile'].values[0])
#     else: 
#         return 0
 

def data_combine(fil,csv_source,save_file):
    """creates the dataframe of stocks which meet mark's crietrion"""

    print(f"Working with {fil}:\n")
    st_list=list(pd.read_csv(root+fil+".csv")['Symbol'])  #list of files  downloaded
    #print(f"st_list:\n{st_list}")


    #Reading the files
    ticker_list=[]           #store ticker
    current_price=[]         #current price
    rsi=[]                   #rsi list
    sma_50=[]                #sma_5 list
    sma_150=[]               #sma_50 list
    sma_200=[]                #sma_200 list
    week_low=[]             #52 week low
    week_high=[]            #52 week high
    rs_value=[]               #rs value
    sma_200past20=[]          #sma 200 past 20
    sector=[]                #sector
    industry=[]              #industry
    
    j=1
    for f in st_list[:100]:    #read these files

        print(f"{j}/{len(st_list)} {fil} : {f}\n")
        j+=1
        #print("source dir\t",csv_source)
        df=pd.read_csv(csv_source+f+".csv")

      

        cp=df.tail(1)['Close'].values[0]                  #current-price
        print(f"Current Price: {cp}\n")
        rs=df.ta.rsi()[-1:].round(2).values[0]            #rsi
        print(f"rsi: {rs}\n")
        sma50=round(df.tail(50)['Close'].mean(),2)        #sma-50
        print(f"sma-50 {sma50}\n")
        sma150=round(df.tail(150)['Close'].mean(),2)      #sma-150
        print(f"sma-150 {sma150}\n")
        sma200=round(df.tail(200)['Close'].mean(),2)      #sma-200
        print(f"sma-200 {sma200}\n")
        sma200past20=round(df['Close'][:-20].tail(200).mean(),2)      #sma-200 past 20
        print(f"sma-200 past 20 {sma200past20}\n")
        week52_high=df['High'].max()                     #52-week high
        print(f"52 week high: {week52_high}\n")
        week52_low=df['Low'].min()                       #52-week low
        print(f"52 week low: {week52_low}\n")
        
        rsval=get_rs_value(f)                            #rs value
        print(f"rs_value of {f}: {rsval}\n")
        
        #sector 
 
        if cp>sma150 and cp>sma200:
            if(sma150>sma200):
                if(sma200>sma200past20):
                    if(sma50>sma150 and sma50>sma200):
                        if(cp>sma50):
                            if(cp>1.30*week52_low):
                                if(cp<1.25*week52_high):
                                    print(f"{f} meets the criterion!")
                                    print(20*"0000")
                                    ticker_list.append(f)
                                    current_price.append(cp)
                                    rsi.append(rs)
                                    sma_50.append(sma50)
                                    sma_150.append(sma150)
                                    sma_200.append(sma200)
                                    sma_200past20.append(sma200past20)
                                    week_high.append(week52_high)
                                    week_low.append(week52_low)
                                    rs_value.append(rsval)
                                    sector.append(df_rs.loc[df_rs['Ticker']==f]['Sector'].values[0])
                                    industry.append(df_rs.loc[df_rs['Ticker']==f]['Industry'].values[0])
            
        
        
        #create the pandas dataframe and save it
        mark_df=pd.DataFrame({
            "TICK":ticker_list,
            "CP":current_price,
            "RSI":rsi,
            "SMA50":sma_50,
            "SMA150":sma_150,
            "SMA200":sma_200,
            "SMA200PAST20":sma_200past20,
            "52WKHIGH":week_high,
            "52WKLOW":week_low,
            "RS":rs_value,
            "SECTOR":sector,
            "INDUSTRY":industry
        })
    print("Data Frame:\n") 
    mark_df.to_csv(save_file)
    print(mark_df)
        
        
        
input_files=["nasdaq_mega", "nasdaq_large","nasdaq_medium","nasdaq_small","nasdaq_micro","nasdaq_nano"]
directories=["mega/","large/","medium/","small/","micro/","nano/"]
today=date.today()
today=today.strftime("%m%d%y")
#today='070622'
print(f"Today: {today}\n")
#n=0
for n in range(6):
    #if n==0: continue
    data_combine(input_files[n],source_dir+directories[n],save_dir+input_files[n]+"mark"+today+".csv")


    
    
    


In [ ]:
df_rs

In [ ]:
get_rs_value("NRIM")

In [ ]:
df=pd.read_csv("/home/thakur/stock_information/nasdaq_nano.csv")
            

In [ ]:
df.head()

In [ ]:
st_list=list(df['Symbol'])
st_list


In [ ]:
#df.loc[df['Symbol']=='A']["Sector"].values[0]

In [ ]:
sec_list=[];ind_list=[]
for i in st_list:
    sec=df.loc[df['Symbol']==i]["Sector"].values[0]
    
    ind=df.loc[df['Symbol']==i]["Industry"].values[0]
    sec_list.append(sec);ind_list.append(ind)
    print(f"{i}\t{sec}\t{ind}\n")
    #print(f"Industry: {df.loc[df['Symbol']==i]["Sector"].values[0]}\n")

In [ ]:
sec_list;ind_list